# Double Trouble Pendulum

In [ ]:
from sympy import *
from sympy.physics.mechanics import *
from math import pi
from scipy.integrate import solve_ivp
import numpy as np
from IPython.display import display
mechanics_printing()
import matplotlib.pyplot as plt

## Create Symbols

In [ ]:
q = dynamicsymbols('\\theta_0, \\theta_1')  # Generalized coordinates (angles)
u = dynamicsymbols('\\omega_0, \\omega_1')  # Generalized speeds (rates)
l0, w0, l1, w1 = symbols('l_0, w_0, l_1, w_1')  # Lengths and widths
m0, m1, I0, I1 = symbols('m_0, m_1, I_0, I_1')  # Masses and inertias
a, b, c = symbols('a, b, c')  # Distances relating pivot points and bodies
g, gx, gy = symbols('g, g_x, g_y')  # Gravitational acceleration
t = symbols('t')  # Time

## Create Initial Values and Constant Values

In [ ]:
q0 = 44*pi/180, 44*pi/180  # radians
u0 = 0.0, 0.0  # radians/second
constants = {
    l0: 0.2,  #0.2794,
    w0: 0.0508,
    l1: 0.15,  #0.2286,
    w1: 0.0508,
    m0: 0.1,  # 0.080,
    m1: 0.1,  # 0.066,
    I0: 5e-6,  # 537.6e-6,
    I1: 5e-6,  # 301.6e-6,
    a: 0.2,  # 0.1016,  # Origin to outer pivot
    b: 0.1,  # 0.1397,  # Origin to inner link CG
    c: 0.075,  # 0.1016,  # Outer pivot to outer link CG
    g: 9.8,
    gy: 9.8,
    gx: 0.0
}

## Create Kinematic Equations

In [ ]:
kd = [speed - diff(coord) for coord, speed in zip(q, u)]
display(Matrix(kd))

## Create Frames

In [ ]:
inertial = ReferenceFrame('N')

innerFrame = inertial.orientnew('D', 'axis', [q[0], inertial.z])
outerFrame = inertial.orientnew('E', 'axis', [q[1], inertial.z])

innerFrame.set_ang_vel(inertial, u[0] * inertial.z)
outerFrame.set_ang_vel(inertial, u[1] * inertial.z)

display(outerFrame.dcm(inertial))
display(outerFrame.ang_vel_in(inertial))

## Create Points

In [ ]:
origin = Point('O')
origin.set_vel(inertial, Vector(0))

outerPivot = origin.locatenew('A', a * innerFrame.x)
innerCG = origin.locatenew('B', b * innerFrame.x)
outerCG = outerPivot.locatenew('C', c * outerFrame.x)
innerEnd = innerCG.locatenew('D', 0.5 * l0 * innerFrame.x)
outerEnd = outerCG.locatenew('E', 0.5 * l1 * outerFrame.x)

outerPivot.v2pt_theory(origin, inertial, innerFrame)
innerCG.v2pt_theory(origin, inertial, innerFrame)
outerCG.v2pt_theory(outerPivot, inertial, outerFrame)

display(innerCG.pos_from(origin))
display(innerCG.vel(inertial))
display(outerCG.pos_from(origin))
display(outerCG.vel(inertial))
display(outerCG.pos_from(origin).to_matrix(inertial))

## Create Bodies

In [ ]:
innerBody = Body('IB', masscenter=innerCG, mass=m0, frame=innerFrame, central_inertia=inertia(innerFrame, 0, 0, I0))
outerBody = Body('OB', masscenter=outerCG, mass=m1, frame=outerFrame, central_inertia=inertia(outerFrame, 0, 0, I1))
bodies = innerBody, outerBody

## Create Gravitational Load

In [ ]:
gvector = gx * inertial.x - gy * inertial.y
loads = [
  (innerBody.masscenter, innerBody.mass * gvector),
  (outerBody.masscenter, outerBody.mass * gvector)
]

## Generate Equations of Motion

In [ ]:
kane = KanesMethod(inertial, q, u, kd)
fr, frstar = kane.kanes_equations(bodies, loads)

## Get the Simplified Mass Matrix

In [ ]:
M = simplify(kane.mass_matrix_full)
display(M)

## Get the Simplified Forcing Vector

In [ ]:
F = simplify(kane.forcing_full)
display(F)

## Create the Initial Value Problem (IVP)

In [ ]:
massFunction = lambdify(q+u, M.subs(constants))
forcingFunction = lambdify(q+u, F.subs(constants))
def fun(t, x):
  mass = massFunction(*x)
  force = forcingFunction(*x)
  dxdt = np.linalg.inv(mass) @ force
  return dxdt[:, 0].tolist()

## Solve the IVP

In [ ]:
sol = solve_ivp(fun, [0, 1.75], q0+u0, rtol=1e-9)

## Plot Angles and Rates

In [ ]:
plt.subplot(2, 1, 1);
plt.plot(sol.t, sol.y[:2].transpose());
plt.ylabel('Angle [rad]');
plt.legend([f'${angle}$' for angle in q], ncol=2);

plt.subplot(2, 1, 2);
plt.plot(sol.t, sol.y[2:].transpose());
plt.xlabel('Time [sec]')
plt.ylabel('Rate [rad/s]');
plt.legend([f'${rate}$' for rate in u], ncol=2);

## Plot X-Y Position

In [ ]:
innerPosFcn = lambdify(q, innerEnd.pos_from(origin).to_matrix(inertial).transpose().subs(constants).tolist()[0])
outerPosFcn = lambdify(q, outerEnd.pos_from(origin).to_matrix(inertial).transpose().subs(constants).tolist()[0])

innerPos = np.array([innerPosFcn(*q) for q in sol.y[:2].transpose()])
outerPos = np.array([outerPosFcn(*q) for q in sol.y[:2].transpose()])

plt.plot(innerPos[:, 0], innerPos[:, 1]);
plt.scatter(outerPos[:, 0], outerPos[:, 1], 
            s=8*sol.t*3.14, c=sol.t, cmap='rainbow');
plt.axis('off');
plt.xlim([-0.5, 0.5])
plt.tight_layout;

plt.savefig('links.pdf')